# Pyber Challenge

### 4.3 Loading and Reading CSV files

In [1]:
# Add Matplotlib inline magic command
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd

# File to Load (Remember to change these)
city_data_to_load = "resources/city_data.csv"
ride_data_to_load = "resources/ride_data.csv"

# Read the City and Ride Data
city_data_df = pd.read_csv(city_data_to_load)
ride_data_df = pd.read_csv(ride_data_to_load)

### Merge the DataFrames

In [2]:
# Combine the data into a single dataset
# Combine the data into a single dataset


pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])

# Created to review why step 2 yielded different results; the merge duplicated values.
# pyber_data_df.to_csv("analysis/check_merge.csv")

# Display the data table for preview
pyber_data_df.head()

,city,date,fare,ride_id,driver_count,type
0,Lake Jonathanshire,2019-01-14 10:14:22,13.83,5739410935873,5,Urban
1,South Michelleport,2019-03-04 18:24:09,30.24,2343912425577,72,Urban
2,Port Samanthamouth,2019-02-24 04:29:00,33.44,2005065760003,57,Urban
3,Rodneyfort,2019-02-10 23:22:03,23.44,5149245426178,34,Urban
4,South Jack,2019-03-06 04:28:35,34.58,3908451377344,46,Urban


## Deliverable 1: Get a Summary DataFrame 

In [3]:
#  1. Get the total rides for each city type
total_rides_per_city = pyber_data_df.groupby("type")["ride_id"].count()
total_rides_per_city.head()

type
Rural        125
Suburban     625
Urban       1625
Name: ride_id, dtype: int64

In [4]:
# 2. Get the total drivers for each city type

# I have to use city_data.csv because of merging problems in step 1
# Come back to see if there is a better solution

total_drivers_per_city = city_data_df.groupby("type")["driver_count"].sum()
total_drivers_per_city.head()

type
Rural         78
Suburban     490
Urban       2405
Name: driver_count, dtype: int64

In [5]:
#  3. Get the total amount of fares for each city type
total_fares_by_city = pyber_data_df.groupby("type")["fare"].sum()
total_fares_by_city.head()

type
Rural        4327.93
Suburban    19356.33
Urban       39854.38
Name: fare, dtype: float64

In [6]:
#  4. Get the average fare per ride for each city type. 
avg_fare_per_city = pyber_data_df.groupby("type")["fare"].mean()
avg_fare_per_city.head()

type
Rural       34.623440
Suburban    30.970128
Urban       24.525772
Name: fare, dtype: float64

In [7]:
# 5. Get the average fare per driver for each city type. 
avg_fare_per_driver_by_city = total_fares_by_city/total_drivers_per_city
avg_fare_per_driver_by_city.head()

type
Rural       55.486282
Suburban    39.502714
Urban       16.571468
dtype: float64

In [8]:
#  6. Create a PyBer summary DataFrame. 
summary_data = {"Total Rides":total_rides_per_city,
                "Total Drivers": total_drivers_per_city,
                "Total Fares": total_fares_by_city, 
                "Average Fare per Ride": avg_fare_per_city, 
                "Average Fare per Driver": avg_fare_per_driver_by_city}
pyber_summary_df =  pd.DataFrame(summary_data)
pyber_summary_df

,Total Rides,Total Drivers,Total Fares,Average Fare per Ride,Average Fare per Driver
type,,,,,
Rural,125,78,4327.93,34.623440,55.486282
Suburban,625,490,19356.33,30.970128,39.502714
Urban,1625,2405,39854.38,24.525772,16.571468


In [9]:
#  7. Cleaning up the DataFrame. Delete the index name
pyber_summary_df.index.name = None
pyber_summary_df



,Total Rides,Total Drivers,Total Fares,Average Fare per Ride,Average Fare per Driver
Rural,125,78,4327.93,34.623440,55.486282
Suburban,625,490,19356.33,30.970128,39.502714
Urban,1625,2405,39854.38,24.525772,16.571468


In [10]:
#  8. Format the columns.

summary_data = {"Total Rides":total_rides_per_city,
                "Total Drivers": total_drivers_per_city,
                "Total Fares": total_fares_by_city,
                "Average Fare per Ride": avg_fare_per_city, 
                "Average Fare per Driver": avg_fare_per_driver_by_city["fare"]}
pyber_summary_df =  pd.DataFrame(summary_data)
pyber_summary_df = pyber_summary_df.style.format({"Total Fares":"${:.2f}", "Average Fare per Ride":"${:.2f}", "Average Fare per Driver": "${:.2f}"},precision=2)

pyber_summary_df


KeyError: 'fare'

## Deliverable 2.  Create a multiple line plot that shows the total weekly of the fares for each type of city.

In [ ]:
# 1. Read the merged DataFrame
pyber_data_df

In [ ]:
# 2. Using groupby() to create a new DataFrame showing the sum of the fares 
#  for each date where the indices are the city type and date.
fares_by_date = pyber_data_df.groupby(["type","date"])["fare"].sum()
pd.DataFrame(fares_by_date)

In [ ]:
# 3. Reset the index on the DataFrame you created in #1. This is needed to use the 'pivot()' function.
# df = df.reset_index()

fares_by_date = fares_by_date.reset_index()

In [ ]:
# 4. Create a pivot table with the 'date' as the index, the columns ='type', and values='fare' 
# to get the total fares for each type of city by the date.

fares_pivot = fares_by_date.pivot_table(index="date", columns = "type", values = "fare")
fares_pivot.head()


In [ ]:
# 5. Create a new DataFrame from the pivot table DataFrame using loc on the given dates, '2019-01-01':'2019-04-29'.

jan_april_2019 = fares_pivot['2019-01-01':'2019-04-29']
jan_april_2019


In [ ]:
# 6. Set the "date" index to datetime datatype. This is necessary to use the resample() method in Step 8.
# df.index = pd.to_datetime(df.index)

jan_april_2019.index = pd.to_datetime(jan_april_2019.index)

In [ ]:
# 7. Check that the datatype for the index is datetime using df.info()

jan_april_2019.info()

jan_april_2019.index

In [ ]:
# 8. Create a new DataFrame using the "resample()" function by week 'W' and get the sum of the fares for each week.

jan_april_2019_by_week = pd.DataFrame(jan_april_2019.resample('W').sum())
jan_april_2019_by_week

In [ ]:
#### 8. Using the object-oriented interface method, plot the resample DataFrame using the df.plot() function. 

# Import the style from Matplotlib.
from matplotlib import style
# Use the graph style fivethirtyeight.
style.use('fivethirtyeight')


fig, ax = plt.subplots(figsize=(15,8))
ax.plot(jan_april_2019_by_week, label=["Rural","Suburban","Urban"])
ax.set_title("Total Fare by City Type")
ax.set_ylabel("Fares ($USD)")
ax.set_xlabel("Months")
ax.legend(loc="center")
ax.set_xticklabels(["January","","February","","March","","April"])
plt.savefig("analysis/Total Fare by City Type.png")
